In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Wyświetl kod</a>''')

In [2]:
import ipywidgets as widgets
from ipywidgets import Layout

style = {'description_width': 'initial'}

In [3]:
n = widgets.FloatLogSlider(value=10000, base=10, min=1, max=10, step=1, description='Liczba symulacji', style=style)
kolejka = widgets.BoundedIntText(value=1, min=1, max=34, step=1, description='Wybierz kolejkę:', disabled=False, style=style)

display(n)
display(kolejka)

button = widgets.Button(description='Zatwierdź',disabled=False,tooltip='Confirm',icon='check') 
output = widgets.Output()
display(button,output)

FloatLogSlider(value=10000.0, description='Liczba symulacji', max=10.0, min=1.0, step=1.0, style=SliderStyle(d…

BoundedIntText(value=1, description='Wybierz kolejkę:', max=34, min=1, style=DescriptionStyle(description_widt…

Button(description='Zatwierdź', icon='check', style=ButtonStyle(), tooltip='Confirm')

Output()

In [4]:
strzelone_dom = {
    1: 13,    2: 15,    3: 8,    4: 13,    5: 15,    6: 10,    7: 0,    8: 0,    9: 0,    10: 0,
    11: 0,    12: 0,    13: 0,    14: 0,    15: 0,    16: 0,    17: 0,    18: 0,    19: 0,    20: 0,
    21: 0,    22: 0,    23: 0,    24: 0,    25: 0,    26: 0,    27: 0,    28: 0,    29: 0,    30: 0,    
    31: 0,     32: 0,    33: 0,     34 : 0
}

In [5]:
strzelone_wyjazd = {
    1: 8,    2: 12,    3: 7,    4: 12,    5: 6,    6: 11,    7: 0,    8: 0,    9: 0,    10: 0,
    11: 0,    12: 0,    13: 0,    14: 0,    15: 0,    16: 0,    17: 0,    18: 0,    19: 0,    20: 0,
    21: 0,    22: 0,    23: 0,    24: 0,    25: 0,    26: 0,    27: 0,    28: 0,    29: 0,    30: 0,    
    31: 0,     32: 0,    33: 0,     34 : 0
}

In [6]:
mecze_liga = 3*9 + 8 + 8 + 7

In [7]:
import pandas as pd
import os

def calculate_weighted_stats(w):
    base = r'KolejkiEkstraklasa'
    matchdays = []
    for filename in os.listdir(base):
        if int(filename)<w:
            matchday = pd.read_csv(f"{base}/{filename}", header=None)
            matchdays.append(matchday)
    league = pd.concat(matchdays, ignore_index=True)
    g = open("Statystyki/poissonstats-ekstraklasa", "r", encoding='utf-8')
    clubs = [x.strip().split(",")[0] for x in g]
    
    global home_attack_strength
    global home_defence_strength
    global away_attack_strength
    global away_defence_strength
    home_attack_strength = {}
    home_defence_strength = {}
    away_attack_strength = {}
    away_defence_strength = {}
    
    global avg_strzelone_dom_liga
    global avg_strzelone_wyjazd_liga
    avg_strzelone_dom_liga = sum([k*v/(8*sum(range(1,w))) for k,v in strzelone_dom.items() if k<w])
    avg_strzelone_wyjazd_liga = sum([k*v/(8*sum(range(1,w))) for k,v in strzelone_wyjazd.items() if k<w])
    
    for c in clubs:
        weight = 1
        weights_away = []
        weights_home = []
        has = 0
        hds = 0
        aas = 0
        ads = 0
        for index, row in league.iterrows():
            home = row[0].split('-')[0]
            away = row[0].split('-')[1]
            if c==home:
                has += int(row[1].split(':')[0])*weight
                hds += int(row[1].split(':')[1])*weight
                weights_home.append(weight)
                weight = weight+1
            if c==away:
                aas += int(row[1].split(':')[1])*weight
                ads += int(row[1].split(':')[0])*weight
                weights_away.append(weight)
                weight = weight+1
        home_attack_strength[c] = has / (sum(weights_home)*avg_strzelone_dom_liga)
        home_defence_strength[c] = hds / (sum(weights_home)*avg_strzelone_wyjazd_liga)
        away_attack_strength[c] = aas / (sum(weights_away)*avg_strzelone_wyjazd_liga)
        away_defence_strength[c] = ads / (sum(weights_away)*avg_strzelone_dom_liga)

In [8]:
import numpy as np
from collections import Counter
from IPython.display import clear_output

def display_matchday(a):
    clear_output()
    h = open(f"KolejkiEkstraklasa/{str(a)}", "r", encoding='utf-8')
    
    for x in h:
        home_team = x.strip().split(",")[0].split("-")[0]
        away_team = x.strip().split(",")[0].split("-")[1]
        rng = np.random.default_rng()
        results = []
        home = home_attack_strength[home_team]*away_defence_strength[away_team]*avg_strzelone_dom_liga
        away = home_defence_strength[home_team]*away_attack_strength[away_team]*avg_strzelone_wyjazd_liga
        print(home_team, "vs", away_team)
        print("Siła:", home, away)
        for i in range(0,int(n.value)):
            results.append((rng.poisson(home, 1)[0],rng.poisson(away, 1)[0]))
        print("Prawdopodobieństwo czystego konta gospodarza:", sum(x[1]==0 for x in results)/n.value)
        print("Prawdopodobieństwo czystego konta gościa:", sum(x[0]==0 for x in results)/n.value)
        
        print("Najbardziej prawdopodobne wyniki:\n",Counter(results).most_common(7),"\n\n\n")
    h.close()

In [9]:
def on_confirm_button_clicked(b):
    calculate_weighted_stats(kolejka.value)
    with output:
        display_matchday(kolejka.value)
    
button.on_click(on_confirm_button_clicked)

In [10]:
#home = home_attack_strength["Podbeskidzie"]*away_defence_strength["Wisła"]*avg_strzelone_dom_liga
#away = home_defence_strength["Podbeskidzie"]*away_attack_strength["Wisła"]*avg_strzelone_wyjazd_liga
#results = []
#rng = np.random.default_rng()
#for i in range(0,int(n.value)):
#    results.append((rng.poisson(home, 1)[0],rng.poisson(away, 1)[0]))
#Counter(results).most_common(35)